In [1]:
!pip install flair

     |████████████████████████████████| 450kB 15.0MB/s 
     |████████████████████████████████| 1.1MB 53.1MB/s 
     |████████████████████████████████| 1.3MB 54.7MB/s 
     |████████████████████████████████| 798kB 51.8MB/s 
     |████████████████████████████████| 983kB 49.8MB/s 
     |████████████████████████████████| 71kB 9.7MB/s 
     |████████████████████████████████| 19.7MB 168kB/s 
     |████████████████████████████████| 2.9MB 49.1MB/s 
     |████████████████████████████████| 890kB 42.8MB/s 
  Created wheel for segtok: filename=segtok-1.5.10-cp36-none-any.whl size=25019 sha256=a23f6c6f024ba774a31917cf9f02319534900cf609f53b018f1e59dd3e4dec4f
  Stored in directory: /root/.cache/pip/wheels/b4/39/f6/9ca1c5cabde964d728023b5751c3a206a5c8cc40252321fb6b
  Created wheel for sqlitedict: filename=sqlitedict-1.7.0-cp36-none-any.whl size=14376 sha256=3c2407974a8bc78dad2654e9cd664e0f69fd9fa46999559786ac7a109f912dfc
  Stored in directory: /root/.cache/pip/wheels/cf/c6/4f/2c64a43f041415eb8b8740bd

In [2]:
# Uncomment if you need to download data
!wget https://raw.githubusercontent.com/Rexhaif/ner-dialogues-hackathon/master/data/train.conll
!wget https://raw.githubusercontent.com/Rexhaif/ner-dialogues-hackathon/master/data/dev.conll
!wget https://raw.githubusercontent.com/Rexhaif/ner-dialogues-hackathon/master/data/test.conll

--2021-02-09 15:52:37--  https://raw.githubusercontent.com/Rexhaif/ner-dialogues-hackathon/master/data/train.conll
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 125059 (122K) [text/plain]
Saving to: ‘train.conll’

train.conll         100%[===================>] 122.13K  --.-KB/s    in 0.003s  

2021-02-09 15:52:37 (40.8 MB/s) - ‘train.conll’ saved [125059/125059]

--2021-02-09 15:52:37--  https://raw.githubusercontent.com/Rexhaif/ner-dialogues-hackathon/master/data/dev.conll
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14109 (14K) [text

In [1]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings, CharacterEmbeddings, FlairEmbeddings, TransformerWordEmbeddings

In [2]:
import flair
import torch
import os
# uncomment for cpu-only training
#flair.device = torch.device('cpu')

In [5]:
# 1. get the corpus
corpus: Corpus = ColumnCorpus(
    "./", column_format={0:'text', 1:'ner'},
    train_file='train.conll', dev_file='dev.conll', test_file='test.conll',
)
print(corpus)

2021-02-09 15:53:00,323 Reading data from .
2021-02-09 15:53:00,325 Train: train.conll
2021-02-09 15:53:00,326 Dev: dev.conll
2021-02-09 15:53:00,326 Test: test.conll
Corpus: 1966 train + 219 dev + 547 test sentences


In [11]:
# 2. what tag do we want to predict?
tag_type = 'ner'

# 3. make the tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)
print(tag_dictionary)

# 4. initialize embeddings
embedding_types = [
    TransformerWordEmbeddings('DeepPavlov/rubert-base-cased')
]

embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

Dictionary with 14 tags: <unk>, O, B-SINGER, I-SINGER, B-SONG, B-FILM, I-SONG, I-FILM, B-COMPOSER, I-COMPOSER, B-BOOK, I-BOOK, <START>, <STOP>


In [12]:
# 5. initialize sequence tagger
from flair.models import SequenceTagger

tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type=tag_type,
                                        use_crf=True)

# 6. initialize trainer
from flair.trainers import ModelTrainer

trainer: ModelTrainer = ModelTrainer(tagger, corpus)

# 7. start training
history = trainer.train(
    './models/baseline-charembeddings',
    learning_rate=0.1,
    mini_batch_size=32,
    max_epochs=20
)

2021-02-09 15:58:15,352 ----------------------------------------------------------------------------------------------------
2021-02-09 15:58:15,355 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): TransformerWordEmbeddings(
      (model): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(119547, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0): BertLayer(
              (attention): BertAttention(
                (self): BertSelfAttention(
                  (query): Linear(in_features=768, out_features=768, bias=True)
                  (key): Linear(in_features=768, out_features=768, bias=True)
                  (value): Linea

In [16]:
tagger1 = SequenceTagger(dropout=0.1, embeddings=embeddings,
                         hidden_size=384,
                         tag_dictionary=tag_dictionary,
                         tag_type=tag_type, use_crf=True)

trainer: ModelTrainer = ModelTrainer(tagger1, corpus)

# 7. start training
history = trainer.train(
    './models/rubert_embeddings',
    learning_rate=0.1,
    mini_batch_size=64,
    max_epochs=20
)

2021-02-09 16:12:20,443 ----------------------------------------------------------------------------------------------------
2021-02-09 16:12:20,446 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): TransformerWordEmbeddings(
      (model): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(119547, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0): BertLayer(
              (attention): BertAttention(
                (self): BertSelfAttention(
                  (query): Linear(in_features=768, out_features=768, bias=True)
                  (key): Linear(in_features=768, out_features=768, bias=True)
                  (value): Linea

Попробуем эмбеддинги Андрея Кутузова

In [17]:
!wget http://vectors.nlpl.eu/repository/20/204.zip

--2021-02-09 16:23:19--  http://vectors.nlpl.eu/repository/20/204.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.181
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2562993511 (2.4G) [application/zip]
Saving to: ‘204.zip’

204.zip             100%[===================>]   2.39G  24.2MB/s    in 1m 43s  

2021-02-09 16:25:02 (23.7 MB/s) - ‘204.zip’ saved [2562993511/2562993511]



In [18]:
!unzip 204.zip

Archive:  204.zip
  inflating: meta.json               
  inflating: model.bin               
  inflating: model.txt               
  inflating: README                  


In [22]:
!python -m pip install --upgrade spacy

     |████████████████████████████████| 12.8MB 252kB/s 
     |████████████████████████████████| 1.1MB 58.2MB/s 
     |████████████████████████████████| 9.2MB 40.7MB/s 
     |████████████████████████████████| 460kB 53.9MB/s 
     |████████████████████████████████| 122kB 52.4MB/s 
     |████████████████████████████████| 102kB 15.5MB/s 
  Created wheel for contextvars: filename=contextvars-2.4-cp36-none-any.whl size=7667 sha256=c902324e831bb9dab6aba26ee10b2756b0f06ee3c77b29be4c002e4af737090f
  Stored in directory: /root/.cache/pip/wheels/a5/7d/68/1ebae2668bda2228686e3c1cf16f2c2384cea6e9334ad5f6de
  Created wheel for smart-open: filename=smart_open-3.0.0-cp36-none-any.whl size=107097 sha256=4ce8c520e3e8f8206c69d61c847eee1701e2fb932e8e8fdc35d532e5a4181ae4
  Stored in directory: /root/.cache/pip/wheels/18/88/7c/f06dabd5e9cabe02d2269167bcacbbf9b47d0c0ff7d6ebcb78
Successfully built contextvars smart-open
  Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Suc

In [23]:
!python -m spacy download ru_core_news_lg

2021-02-09 16:33:21.977046: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
     |████████████████████████████████| 515.3MB 33kB/s 
     |████████████████████████████████| 61kB 5.1MB/s 
     |████████████████████████████████| 8.2MB 15.7MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_lg')


In [25]:
import spacy
nlp = spacy.load("ru_core_news_lg")

In [39]:
def load_sentences(file):
  with open(file,'r',encoding='utf-8') as inp:
    text = inp.read()
  
  sents = text.split('\n\n')

  output = []

  for sent in sents:
    out_words = []
    out_tags = []
    for line in sent.splitlines():
      try:
        word, tag = line.split('\t')
      except:
        word, tag = line.split()
      out_words.append(word)
      out_tags.append(tag)
    output.append((out_words, out_tags))

  return output

In [31]:
os.listdir()

['.config',
 'meta.json',
 'model.txt',
 'model.bin',
 'dev.conll',
 '204.zip',
 'README',
 'models',
 'test.conll',
 'train.conll',
 'sample_data']

In [47]:
from tqdm import tqdm_notebook

In [50]:
def parse_sentences(inp_file):
  sents = load_sentences(inp_file)
  for i in tqdm_notebook(range(len(sents)), total=len(sents)):
    tokens, tags = sents[i]
    sent = [token.lemma_+'_'+token.pos_ for token in nlp(' '.join(tokens))]
    assert len(sent) == len(tags)
    sents[i] = (sent, tags)
  return sents

In [52]:
s = parse_sentences('train.conll')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [53]:
s = parse_sentences('test.conll')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [55]:
s = parse_sentences('dev.conll')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [67]:
def save_parsed(inp_file, outp_file):
  sents = parse_sentences(inp_file)

  with open(outp_file, 'w', encoding='utf-8') as outp:
    for sent in sents:
      for token, tag in zip(*sent):
        outp.write(token+' '+tag)
        outp.write('\n')
      outp.write('\n')

In [68]:
save_parsed('train.conll','train_parsed.conll')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [70]:
save_parsed('dev.conll','dev_parsed.conll')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [71]:
save_parsed('test.conll','test_parsed.conll')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [3]:
# 1. get the corpus
corpus1: Corpus = ColumnCorpus(
    "./", column_format={0:'text', 1:'ner'},
    train_file='train_parsed.conll', dev_file='dev_parsed.conll', test_file='test_parsed.conll',
)
print(corpus1)

2021-02-09 17:18:00,690 Reading data from .
2021-02-09 17:18:00,693 Train: train_parsed.conll
2021-02-09 17:18:00,694 Dev: dev_parsed.conll
2021-02-09 17:18:00,696 Test: test_parsed.conll
Corpus: 1966 train + 219 dev + 547 test sentences


In [73]:
os.listdir()

['.config',
 'meta.json',
 'dev_parsed.conll',
 'model.txt',
 'train_parsed.conll',
 'model.bin',
 'test_parsed.conll',
 'dev.conll',
 '204.zip',
 'README',
 'models',
 'test.conll',
 'train.conll',
 'sample_data']

In [74]:
import gensim

word_vectors = gensim.models.KeyedVectors.load_word2vec_format('model.txt', binary=False)
word_vectors.save('glove.txt')

In [5]:
# 2. what tag do we want to predict?
tag_type = 'ner'

# 3. make the tag dictionary from the corpus
tag_dictionary = corpus1.make_tag_dictionary(tag_type=tag_type)
print(tag_dictionary)

# 4. initialize embeddings
embedding_types = [
    WordEmbeddings('glove.txt')
]

embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

Dictionary with 14 tags: <unk>, O, B-SINGER, I-SINGER, B-SONG, B-FILM, I-SONG, I-FILM, B-COMPOSER, I-COMPOSER, B-BOOK, I-BOOK, <START>, <STOP>


In [6]:
# 5. initialize sequence tagger
from flair.models import SequenceTagger

tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type=tag_type,
                                        use_crf=True)

# 6. initialize trainer
from flair.trainers import ModelTrainer

trainer: ModelTrainer = ModelTrainer(tagger, corpus1)

# 7. start training
history = trainer.train(
    './models/baseline-charembeddings',
    learning_rate=0.1,
    mini_batch_size=32,
    max_epochs=20
)

2021-02-09 17:19:12,306 ----------------------------------------------------------------------------------------------------
2021-02-09 17:19:12,307 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings('glove.txt')
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=300, out_features=300, bias=True)
  (rnn): LSTM(300, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=14, bias=True)
  (beta): 1.0
  (weights): None
  (weight_tensor) None
)"
2021-02-09 17:19:12,308 ----------------------------------------------------------------------------------------------------
2021-02-09 17:19:12,313 Corpus: "Corpus: 1966 train + 219 dev + 547 test sentences"
2021-02-09 17:19:12,314 ----------------------------------------------------------------------------------------------------
2021-02-09 17:19:12,316 Parameters:
2021-02-09 17:19:12,317  - learn